In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import cv2
import numpy as np
from ROIPoolingLayer import RoiPoolingConv
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


def make_RPN(anchors_num=9):
    feature_map_title = layers.Input(shape=(None, None, 2048))

    k = anchors_num
    x = layers.Conv2D(512, kernel_size=(3, 3))(feature_map_title)
    reg = layers.Conv2D(4*k, kernel_size=(1, 1), activation='sigmoid')(x)
    scores = layers.Conv2D(1*k, kernel_size=(1, 1))(x)

    model = Model(inputs=[feature_map_title], outputs=[scores, reg])

    # model.summary()
    return model

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs Available:  2


In [2]:
img = cv2.imread('test.jpg')
h, w, _ = img.shape

In [3]:
print(h,w)

720 1280


In [4]:
img = cv2.resize(img, (224, 224))
img = np.expand_dims(img, 0)

In [5]:
print(img.shape)

(1, 224, 224, 3)


In [6]:
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
# with tf.device('/device:GPU:0'):
model_rpn = make_RPN()
resnet = tf.keras.applications.ResNet50(include_top=False)
feature_map = resnet.predict(img)

print(feature_map.shape)

(1, 7, 7, 2048)


In [7]:
score, reg = model_rpn(feature_map)

In [8]:
print(score.shape)
print(reg.shape)

(1, 5, 5, 9)
(1, 5, 5, 36)


In [9]:
print(reg.shape)

(1, 5, 5, 36)


In [10]:
import models

In [11]:
roi_input = layers.Input(shape=(None, 4))

channels_last


TypeError: 'NoneType' object is not subscriptable

In [19]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import TimeDistributed, Dense, Flatten, Dropout, AveragePooling2D
from ROIPoolingLayer import RoiPoolingConv

def classifier(base_layers, input_rois, num_rois, nb_classes=21, trainable=False):
    print(base_layers)
    print(input_rois)
    print(num_rois)
    
    pooling_regions = 14
    # densenet output channels are 1024..
    input_shape = (num_rois, 14, 14, 1024)

    # from vgg version..
    out_roi_pool = RoiPoolingConv(pooling_regions, num_rois)([
        base_layers, input_rois])

    out_roi_pool = TimeDistributed(AveragePooling2D(
        (7, 7)), name='avg_pool')(out_roi_pool)
    out = TimeDistributed(Flatten(name='flatten'))(out_roi_pool)
    out = TimeDistributed(Dense(4096, activation='relu', name='fc1'))(out)
    out = TimeDistributed(Dropout(0.5))(out)
    out = TimeDistributed(Dense(4096, activation='relu', name='fc2'))(out)
    out = TimeDistributed(Dropout(0.5))(out)

    out_class = TimeDistributed(Dense(nb_classes, activation='softmax',
                                      kernel_initializer='zero'), name='dense_class_{}'.format(nb_classes))(out)
    # note: no regression target for bg class
    out_regr = TimeDistributed(Dense(4 * (nb_classes-1), activation='linear',
                                     kernel_initializer='zero'), name='dense_regress_{}'.format(nb_classes))(out)

    return [out_class, out_regr]


In [21]:
print(resnet.layers)
# classs = classifier(resnet,roi_input,32,trainable=True)

[<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f08ccf48990>, <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f08cc31fbd0>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f08cc31ff10>, <tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7f08cc2b3e90>, <tensorflow.python.keras.layers.core.Activation object at 0x7f08cc2b9110>, <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f08cc2c0610>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f08cc2c03d0>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f08cc28e890>, <tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7f08cc29a1d0>, <tensorflow.python.keras.layers.core.Activation object at 0x7f08cc242950>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f08cc1ec2d0>, <tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7f08cc1f3d10

In [36]:
def make_RPN2(anchors_num=9):
    feature_map_title = layers.Input(shape=(None, None, 2048))

    k = anchors_num
    x = layers.Conv2D(512, kernel_size=(3, 3))(feature_map_title)
    reg = layers.Conv2D(4*k, kernel_size=(1, 1), activation='sigmoid')(x)
    scores = layers.Conv2D(1*k, kernel_size=(1, 1))(x)
    
    print(reg.tensor)

    model = Model(inputs=[feature_map_title], outputs=[scores, reg])

    # model.summary()
    return model

In [37]:
test = make_RPN2()

AttributeError: 'Tensor' object has no attribute 'tensor'

In [39]:
test.outputs

[<tf.Tensor 'conv2d_8/Identity:0' shape=(None, None, None, 9) dtype=float32>,
 <tf.Tensor 'conv2d_7/Identity:0' shape=(None, None, None, 36) dtype=float32>]

In [40]:
test.inputs

[<tf.Tensor 'input_5:0' shape=(None, None, None, 2048) dtype=float32>]

In [41]:
print(resnet)

In [42]:
resnet.outputs[0]

[<tf.Tensor 'conv5_block3_out/Identity:0' shape=(None, None, None, 2048) dtype=float32>]

In [44]:
resnet.inputs

[<tf.Tensor 'input_2:0' shape=(None, None, None, 3) dtype=float32>]

In [45]:
resnet.layers